# PyTorch - o co chodzi?

#### Uwaga: nie edytujcie i nie odpalajcie komórek z kodem. Są tylko dla przykładu i się nie wykonają.

Przede wszystkim, PyTorch wprowadza:

- n-wymiarową strukturę Tensor, praktcznie identyczna do numpy'owej 'ndarray',
ale z supportem dla obliczeń na GPU

- Automatyczne różniczkowanie, które pozwala bardzo uprościć zapis liczenia
gradientów i aktualizacji parametrów sieci neuronowych

- moduły takie jak torch.nn czy torch.F, zawierające gotowe klasy i metody do
budowania sieci (np. gotowe 'layery', funkcje aktywacji)

## 0. Prerekwizyty

Warto byłoby wcześniej wiedzieć chociaż minimalnie o:
- Numpy. Co to ndarray, podstawowe operacje, jak można wybierać z tych struktur dane.
- Sieci neuronowe - jak wygląda forward pass, o co chodzi w backpropagation (opcjonalnie)
- Co to gradient, a co to pochodna cząstkowa, ważne, żeby to rozróżniać, bo często używa się jako skrót myślowy jednego zwrotu zamiast drugiego

## 1. Tensory

Podstawowe 'twory' PyTorcha, którymi posługujemy się tak, jak ndarray's, ale
w łatwy sposób możemy używać GPU do obliczeń z nimi związanych.

- Tensory nic nie wiedzą o deep learning, pochodnych itd. Zamysł twórców jest taki, że można używać PyTorcha i Tensorów do innych rzeczy, w których jest dużo operacji macierzowych, a deep learning to tylko jedno z zastosowań, dla którego istnieje cała reszta modułów.

- Większość operacji można robić na 2 sposoby

(x i y to Tensory, za add można tu wstawić w zasadzie wszystkie tego typu operacje)

In [ ]:
# Note - przykłady w kodzie są read-only, nie wykonają się w notebooku, nie ma importów itd.

torch.add(x, y)
x.add_(y) # zmienia in-place
x.add(y)  # zwraca nowy tensor

- Numpy bridges

In [ ]:
# z numpy do tensora:
y = torch.from_numpy(x)

# z tensora do numpy:
z = y.numpy()

Wszystkie operacje (docs bez którego nie przetrwa nikt podczas pisania kodu PyTorch :D):

http://pytorch.org/docs/master/torch.html

## Autograd i Variables
http://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html

Jeden wrapper dla twórców PyTorcha to za mało, więc zrobili jeszcze wrapper
na Tensory - Variable z pakietu autograd:

In [ ]:
x = torch.autograd.Variable(torch.FloatTensor(np.array([0.1, 0.2])))

### Autograd to moduł odpowiadający za automatyczne liczenie pochodnych.
Dla zarejestrowanej Variable 'zapamiętuje' historię operacji, które brały udział w jej stworzeniu w postaci grafu.

### Każda Variable posiada pola:
- .data - x.data zwraca tensor z danymi jakie 'opakowuje'
- .grad - wartość gradientu dla tej zmiennej
- .grad_fn - funkcja która utworzyła tę zmienną. Jeżeli user tworzy sam nową zmienną, to jest to None, jeżeli np. dodano dwie inne zmienne, aby otrzymać tę zmienną, to jest to funkcja dodawania itd. 
- ! Uwaga - ta funkcja (jest to ofc jakiś obiekt) ma w środku oczywiście odwołania do zmiennych, które były parametrami tej funkcji, więc może z nich odczytać funkcje tworzące te parametry - powstaje taka 'rekurencja' której wynikiem może być cały 'graf'

### Jak tego używać?

Gdy rejestrujemy/tworzymy zmienne, to po to, aby wykorzystać je do obliczenia jakiejś wartości - <b>u nas to będzie 'loss'</b>, wartość funkcji która mówi jak bardzo nasza sieć 'nie ma racji'. 
<b>Chcemy minimalizować tę funkcję</b>.

- Po pewnych obliczeniach - u nas związanych z alg. Q-Learning - otrzymujemy Variable 'loss'
- PyTorch trzyma całą 'historię' operacji, które były potrzebne do otrzymania tej wartości (możemy sobie wręcz wyobrazić, że trzyma taki jeden, końcowy duży wzór na funkcję loss, gdzie zmiennymi są wszystkie wartości, których użyliśmy po drodze) 
- Wykonując loss.backward(), wywołujemy wewnętrzny mechanizm, który oblicza nam gradient, czyli pochodne cząstkowe względem wszystkich parametrów, które brały udział w obliczeniu tej funkcji.
- Każda Variable, która brałą udział w obliczeniu loss, ma teraz w polu .grad nową wartość - <b>wartość pochodnej cząstkowej tego 'finalnego wzoru' względem tej Variable</b>

Czyli przykładowo:

In [ ]:
w = Variable(torch.Tensor(np.array([0.5])))
x = w * w
y = Variable(torch.Tensor(np.array([2.0])))

z = x*y

z.backward()

# z = x * y = w^2 * y
# pochodna cząstkowa 'funkcji' z po parametrze w jest równa 2w * y, czyli 1.0 * 2.0 = 2.0
# pochodna cząstkowa z po parametrze x równa jest y, czyli 2.0
# pochodna cząstkowa z po parametrze y równa jest x, czyli 1.0
# zatem:
# w.grad = [2.0]
# x.grad = [2.0] (to jest Variabler, a nie pojedyńcza liczba, bo w domyśle x i y to mogą być macierze ofc, 
                # przykład jest dla 1-elemtowych 1d Variables)
# y.grad = [1.0]

# Uwaga - tak naprawdę to domyślnie nic się nie zmieni w polach grad - patrz niżej!

Co nam to daje? Jeżeli zarejestrowanymi zmiennymi były wagi i biasy sieci neuronowej,
to dzieki .backward() mamy obliczony gradient, w kierunku którego aktualizujemy wartości
tych parametrów, 'ulepszając sieć'.

### Dodatkowo: tworząc zmienną, można powiedzieć parametrem, czy chcemy, aby pytorch liczył dla niej gradient (domyślnie FALSE!):

In [ ]:
y = Variable(tensor1, requires_grad=True)
x = Variable(tensor2, requires_grad=False)

Z = x + y

Z.backward() # y.grad zostanie zaktualizowane, x.grad nie.

## Moduł nn - neural netowork

- Moduł ten korzysta z modułu autograd, aby definiowane modele były łatwe w obsłudze jeżeli chodzi o proces uczenia. Dokładniej - my nie tworzymy bezpośrednio Variables które są wagami i biasami sieci - to dzieje się pod spodem - <b>i co ważne - te parametry są tworzone z flagą requires_grad=True</b>

- Poniżej prosta definicja sieci z hidden layerem. Jak widać wykorzystujemy moduł nn do prostej definicji layerów np. Linear (używamy tych layerów jak funkcje! np. x = self.fc1(x)), oraz modułu F do różnego rodzaju funkcji aktywacji na poszczególnych warstwach. 

In [ ]:
class Net(nn.Module):
    """ Prosta sieć w PyTorch (3-layer)
    """
    def __init__(self, i_size, h_size, o_size):
        super().__init__()
        self.fc1 = nn.Linear(i_size, h_size)
        self.fc2 = nn.Linear(h_size, o_size)

    def forward(self, x):
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        return F.softmax(x, dim=1)

## Mamy sieć, wiemy coś o Variable i aurograd. Jak uczyć?

- Brakuje nam jeszcze jednej rzeczy. Załóżmy, że mamy zdefiniowaną sieć, przekazaliśmy input ze zbioru uczącego, na podstawie wyniku i wartości oczekiwanej mamy policzony loss, a z loss.backward() jest policzony gradient.

- Musimy teraz zaktualizować parametry zgodnie z tym gradientem. 

- Możemy to zrobić Pythonowo:

In [ ]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate) # jak widać f.grad to Variable, 
                                             # więc wołamy .data przed operacją, aby 'wyłuskać' tensor)

- Ale PyTorch ma jeszcze jedną świetną część. Jest to moduł <b>optim</b>,  który pozwala na przeprowadzanie tych aktualizacji automatycznie, a co więcej, zgodnie z różnymi algorytmami opracowanymi przez badaczy. Ten powyżej wykorzystuje stałe learning_rate, a istnieje wiele metodyk obniżania/dostosowywania tego parametru wraz z uczeniem się, np. Adam, Adagrad itd. 
- Przykładowo:

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)
# optimizer = optim.Adam(net.parameters()) itd. itp.

optimizer.zero_grad()               # zerowanie gradientu
output = net(input)                 # forward pass sieci
loss = criterion(output, target)    # obliczenie loss
loss.backward()                     # obliczamy gradient
optimizer.step()                    # aktualizujemy parametry

Uwaga: Warto zauważyć, że .backward() nie ustala wartości .grad na nowo, tylko 'appenduje', dodając do obecnej wartości nowy gradient, więc przed każdą nową iteracją optimizer.zero_grad() resetuje te wartości. 

# TBC - bardziej skomplikowane operacje na Tensorach, jak rozumieć ich size, operacje z parametrem dim, gather(), squeeze() itd.